In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch as T
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        return [unicodeToAscii(line.strip()) for line in some_file]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Arabic', 'Chinese', 'Czech', 'Dutch', 'English', 'French', 'German', 'Greek', 'Irish', 'Italian', 'Japanese', 'Korean', 'Polish', 'Portuguese', 'Russian', 'Scottish', 'Spanish', 'Vietnamese']
O'Neal


In [3]:
device = T.device('cuda' if T.cuda.is_available() else 'cpu')

In [4]:
import random
# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [5]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = T.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = T.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return T.LongTensor(letter_indexes)

In [6]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [7]:
#Create the memory network
class MemNet2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, memory_size,
                 forget_rate=0.5, device='cpu'):
        super(MemNet2, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.memory_size = memory_size
        self.forget_rate = forget_rate
        self.device = device
        # Make the memory cell
        self.M_l = nn.Parameter(T.zeros(self.memory_size)).to(self.device)
        #Make the model following the architecture above
        self.f_t = nn.Linear(input_size + n_categories + hidden_size, memory_size)
        self.f_t_sigmoid = nn.Sigmoid()
        self.g_t = nn.Linear(input_size + n_categories + hidden_size, memory_size)
        self.g_t_sigmoid = nn.Sigmoid()
        self.o_t = nn.Linear(input_size + n_categories + hidden_size, memory_size)
        self.o_t_tanh = nn.Tanh()
        self.d_t = nn.Linear(input_size + n_categories + hidden_size, hidden_size)
        self.d_t_sigmoid = nn.Sigmoid()
        self.Out = nn.Linear(memory_size + hidden_size, output_size)
        self.Out_softmax = nn.LogSoftmax(dim=1)
        self.L_t = nn.Linear(memory_size + hidden_size, memory_size)
        self.L_t_tanh = nn.Tanh()
    def forward(self, category, x_t, h_t_prev, c_t_prev):
        Cat_inp = T.cat((category, x_t, h_t_prev), dim=1)
        f_t = self.f_t_sigmoid(self.f_t(Cat_inp))
        g_t = self.g_t_sigmoid(self.g_t(Cat_inp))
        o_t = T.tanh(self.o_t(Cat_inp))
        d_t = self.d_t_sigmoid(self.d_t(Cat_inp))
        F_C_t = f_t * c_t_prev
        D_C_t = g_t * o_t
        S_C_t = F_C_t + D_C_t
        C_t = F.tanh(self.M_l * S_C_t)
        Cat_L = T.cat((C_t, d_t), dim=1)
        L_t = T.tanh(self.L_t(Cat_L))
        self.M_l = (self.M_l.add(self.forget_rate * L_t)).clone()
        output = self.Out_softmax(self.Out(Cat_L))
        h_t = C_t * d_t
        return output, h_t, C_t
    def init_hidden(self):
        return (T.zeros(1, self.hidden_size, device=self.device),
                T.zeros(1, self.memory_size, device=self.device))
    def reset_memory(self):
        self.M_l = nn.Parameter(T.zeros(self.memory_size)).to(self.device)

In [8]:
#Create the memory network
class MemNet3(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, memory_size,
                 forget_rate=0.5, device='cpu'):
        super(MemNet3, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.memory_size = memory_size
        self.forget_rate = forget_rate
        self.device = device
        # Make the memory cell
        self.M_l = nn.Parameter(T.zeros(self.memory_size)).to(self.device)
        #Make the model following the architecture above
        self.f_t = nn.Linear(input_size + n_categories + hidden_size, memory_size)
        self.f_t_sigmoid = nn.Sigmoid()
        self.g_t = nn.Linear(input_size + n_categories + hidden_size, memory_size)
        self.g_t_sigmoid = nn.Sigmoid()
        self.y_t = nn.Linear(input_size + n_categories + hidden_size, memory_size)
        self.y_t_sigmoid = nn.Sigmoid()
        self.d_t = nn.Linear(input_size + n_categories + hidden_size, hidden_size)
        self.Out = nn.Linear(memory_size + hidden_size, output_size)
        self.Out_softmax = nn.LogSoftmax(dim=1)
        self.L_t = nn.Linear(memory_size + hidden_size, memory_size)
        self.L_t_tanh = nn.Tanh()
    def forward(self, category, x_t, h_t_prev, c_t_prev):
        Cat_inp = T.cat((category, x_t, h_t_prev), dim=1)
        f_t = self.f_t_sigmoid(self.f_t(Cat_inp))
        g_t = self.g_t_sigmoid(self.g_t(Cat_inp))
        o_t = T.tanh(self.d_t(Cat_inp))
        y_t = self.y_t_sigmoid(self.y_t(Cat_inp))
        F_C_t = f_t * c_t_prev
        G_D_t = g_t * o_t
        F_G_D_t = F_C_t + G_D_t
        C_t = self.M_l * F_G_D_t
        L_t = T.tanh(self.L_t(T.cat((C_t, y_t), dim=1)))
        self.M_l = (self.M_l.add(self.forget_rate * L_t)).clone()
        h_t = T.tanh(C_t) * y_t
        output = self.Out_softmax(self.Out(T.cat((C_t, h_t), dim=1)))
        #print (output.shape)
        return output, h_t, C_t
    def init_hidden(self):
        return (T.zeros(1, self.hidden_size, device=self.device),
                T.zeros(1, self.memory_size, device=self.device))
    def reset_memory(self):
        self.M_l = nn.Parameter(T.zeros(self.memory_size)).to(self.device)

In [22]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device='cpu'):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.LSTM = nn.LSTM(input_size + n_categories, hidden_size, batch_first=True, bidirectional=False)
        self.Out = nn.Linear(hidden_size, output_size)  # output layer
        self.Out_softmax = nn.LogSoftmax(dim=1)
        self.device = device
    def forward(self,n_categories, x, h_t_prev, c_t_prev):
        #print (x.shape)
        Cat_inp = T.cat((n_categories,x), dim=1)
        output, (h_t, c_t) = self.LSTM(Cat_inp, (h_t_prev, c_t_prev))
        #print (output.shape)
        output = self.Out_softmax(self.Out(output))
        return output, h_t, c_t
    def init_hidden(self):
        return (T.zeros(1, self.hidden_size, device=self.device),
                T.zeros(1, self.hidden_size, device=self.device))


In [17]:
"""Cat_inp = T.cat((category, x_t, h_t_prev), dim=1)
        f_t = self.f_t(Cat_inp)
        f_t_sigmoid = T.sigmoid(f_t)
        g_t = self.g_t(Cat_inp)
        g_t_sigmoid = T.sigmoid(g_t)
        o_t = self.y_t(Cat_inp)
        o_t = T.tanh(self.d_t(Cat_inp))
        y_t = self.y_t_sigmoid(self.y_t(Cat_inp))
        F_C_t = f_t * c_t_prev
        G_D_t = g_t * o_t
        F_G_D_t = F_C_t + G_D_t
        C_t = self.M_l * F_G_D_t
        L_t = T.tanh(self.L_t(T.cat((C_t, y_t), dim=1)))
        self.M_l = (self.M_l.add(self.forget_rate * L_t)).clone()
        h_t = T.tanh(C_t) * y_t
        output = self.Out_softmax(self.Out(T.cat((C_t, h_t), dim=1)))
        #print (output.shape)
        return output, h_t, C_t"""

'Cat_inp = T.cat((category, x_t, h_t_prev), dim=1)\n        f_t = self.f_t(Cat_inp)\n        f_t_sigmoid = T.sigmoid(f_t)\n        g_t = self.g_t(Cat_inp)\n        g_t_sigmoid = T.sigmoid(g_t)\n        o_t = self.y_t(Cat_inp)\n        o_t = T.tanh(self.d_t(Cat_inp))\n        y_t = self.y_t_sigmoid(self.y_t(Cat_inp))\n        F_C_t = f_t * c_t_prev\n        G_D_t = g_t * o_t\n        F_G_D_t = F_C_t + G_D_t\n        C_t = self.M_l * F_G_D_t\n        L_t = T.tanh(self.L_t(T.cat((C_t, y_t), dim=1)))\n        self.M_l = (self.M_l.add(self.forget_rate * L_t)).clone()\n        h_t = T.tanh(C_t) * y_t\n        output = self.Out_softmax(self.Out(T.cat((C_t, h_t), dim=1)))\n        #print (output.shape)\n        return output, h_t, C_t'

In [23]:
MemNet = LSTM(n_letters, 128, n_letters, device=device).to(device)

In [24]:
criterion = nn.NLLLoss()

learning_rate = 0.05

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden, recall = MemNet.init_hidden()
    MemNet.zero_grad()
    
    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden, recall = MemNet(category_tensor.to(device), input_line_tensor[i].to(device), hidden, recall)
        l = criterion(output, target_line_tensor[i].to(device))
        loss += l

    loss.backward()

    for p in MemNet.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)

In [25]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [26]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with T.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category).to(device)
        input = inputTensor(start_letter).to(device)
        hidden,recall = MemNet.init_hidden()
        output_name = start_letter

        for i in range(max_length):
            output, hidden, recall = MemNet(category_tensor.to(device), input[0].to(device), hidden.to(device), recall.to(device))
            output.to(device)
            hidden.to(device)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'RUS')

samples('German', 'GER')

samples('Spanish', 'SPA')

samples('Chinese', 'CHI')

RKKKKKKKKYKKKKKKKKKKK
UKKKKKKKKKKKKKKKKKKKK
S;KKKKKKKKKKKKKKKKKKK
G;KKKKYKKKKYKKKKYKKKK
EzEzKKKYKKKKYKKKKYKKK
RKKYKKKKYKKKKYKKKKYKK
S;YYzYzzYzzAzAzAzAzAz
PzYzYzzAzAzAzAzAzAzAz
AzYzYzzAzAzAzAzAzAzAz
C;AAAAAAAAAAAAAAAAAAA
HYYKYKYKYKYKYKYKYKYKY
IYKYKYKYKYKYKYKYKYKYK


In [ ]:
n_iters = 100000
print_every = 200
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    if loss > 1000:
        print("Loss is too high: ", loss)
        print("Due to this output: ", output)
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))
        samples('Irish', 'RUS')

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 2s (200 0%) 2.5433
Riaghain
Uarah
Salaman


KeyboardInterrupt: 

In [ ]:
#save
T.save(MemNet.state_dict(), './modelMem.pth')

In [ ]:
#Load model
#MemNet.load_state_dict(T.load('./modelMem.pth'))

In [ ]:
samples('Scottish', "ABCDEFGHIJKLMNOPQRSTUVWXYZ")

Allar
Brack
Chart
Duckerson
Ester
Frant
Grack
Harter
Ister
Jackson
Krack
Leclard
Murthe
Ningerson
Ollard
Pinter
Qurrin
Rimisson
Silkar
Track
Urant
Vingerson
Willer
Xingerson
Youller
Zhart
